In [15]:
import pandas as pd



In [16]:
xls = pd.ExcelFile('data_sets/2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx')

# 获取所有工作表的名称
sheet_names = xls.sheet_names
sheet_names

['Data Info Panel',
 'Metadata',
 'AllData',
 'Morwell East_10017',
 'Altona North_10006',
 'Dandenong_10022',
 'Point Cook_10005',
 'Brooklyn_10213',
 'Geelong South_10107',
 'Mooroolbark_10136',
 'Footscray_10451',
 'Traralgon_10011',
 'Alphington_10001',
 'Churchill_10219',
 'Melton_10169',
 'Moe_10218',
 'Newborough_10352',
 'Campbellfield_10425',
 'Box Hill_10042',
 'Melbourne CBD_10239',
 'Brighton_10007',
 'Morwell South_10217',
 'Bendigo_10432']

In [17]:
df_mel = pd.read_excel('data_sets/2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Melbourne CBD_10239')
# df_bh= pd.read_excel('2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Box Hill_10042')
# df_melton = pd.read_excel('2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Melton_10169')
# # df_Dandenong = pd.read_excel('2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Dandenong_10022')
# # df_PointCook = pd.read_excel('2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Point Cook_10005')
# df_Geelong = pd.read_excel('2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Geelong South_10107')
# df_Bendigo= pd.read_excel('2022_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', sheet_name = 'Bendigo_10432')

In [18]:
# df_Geelong = df_Geelong.rename(columns={'averageValue': 'BPM2.5'})

In [19]:
# df_mel['datetime_local'] = pd.to_datetime(df_mel['datetime_local'])
# df_mel['datetime_local'] = df_mel['datetime_local'].dt.strftime('%Y-%m-%d-%H')
# df_mel

In [20]:
def preprocess_dataframe(df):
    # 将日期时间列转换为日期，并丢弃时间部分
    
    df['datetime_local'] = pd.to_datetime(df['datetime_local'])
    df['datetime_local'] = df['datetime_local'].dt.strftime('%Y-%m-%d-%H')
    # 根据日期进行分组，并计算每天的平均bpm2.5值
    new_df = df.groupby(['datetime_local', 'location_name'])['BPM2.5'].mean().reset_index()
    new_df2 = new_df.dropna()
    return new_df2

In [21]:
def preprocess_dataframe2(df):
    # 将日期时间列转换为日期，并丢弃时间部分
    df_mel['datetime_local'] = pd.to_datetime(df_mel['datetime_local'])
    df_mel['datetime_local'] = df_mel['datetime_local'].dt.strftime('%Y-%m-%d-%H')
    df= df.dropna()
    # 根据日期进行分组，并计算每天的平均bpm2.5值
    new_df = df.groupby(['datetime_local', 'location_name'])['SBPM25'].mean().reset_index()
    
    df2 = new_df.rename(columns={'SBPM25': 'BPM2.5'})
    return df2

In [22]:
df_mel2 = preprocess_dataframe(df_mel)
# df_bh2 = preprocess_dataframe2(df_bh)
# df_melton = preprocess_dataframe2(df_melton)
# df_Bendigo2 = preprocess_dataframe2(df_Bendigo)
# df_Geelong2 = preprocess_dataframe(df_Geelong)

In [23]:
df_mel2

,datetime_local,location_name,BPM2.5
1,2022-01-01-01,Melbourne CBD,11.834
2,2022-01-01-02,Melbourne CBD,16.082
3,2022-01-01-03,Melbourne CBD,14.921
4,2022-01-01-04,Melbourne CBD,13.251
5,2022-01-01-05,Melbourne CBD,4.930
...,...,...,...
8754,2022-12-31-19,Melbourne CBD,10.708
8755,2022-12-31-20,Melbourne CBD,4.900
8756,2022-12-31-21,Melbourne CBD,8.210
8757,2022-12-31-22,Melbourne CBD,12.359


In [24]:
# concatenated_df = pd.concat([df_mel2, df_bh2,df_melton, df_Bendigo2, df_Geelong2 ], axis=0, ignore_index=True)

# # 打印连接后的数据框的前几行
# concatenated_df

In [25]:
# extract mel 1 hour data from real time data
def find_mel_1hr_data(filename):
    air_data = pd.read_csv('air_data_realtime.csv')
    filtered_df = air_data [air_data ['timeSeriesName'] == '1HR_AV']
    mel_df = filtered_df[filtered_df['sit_name'] == 'Melbourne CBD']
    return mel_df

# preprocess the extracted data
def preprocess_new_air(df):
    air_data = df

    target_cols = ['sit_name', 'until', 'averageValue']
    air_data2 = air_data.filter(items=target_cols)

    air_data2['until'] = pd.to_datetime(air_data2['until'])
    air_data2['datetime_local'] = air_data2['until'].dt.strftime('%Y-%m-%d-%H')


    # Change col name
    air_data2 = air_data2.rename(columns={'sit_name': 'location_name'})
    air_data2 = air_data2.rename(columns={'averageValue': 'BPM2.5'})

    # change col order
    new_order = ['datetime_local', 'location_name', 'BPM2.5']
    air_data2 = air_data2.reindex(columns=new_order)
    
    # air_data2.drop(columns=['until'], inplace=True)

    return air_data2

# add the new data to the static data set
def concate_new_with_old(df, old_df):
    new_df =  preprocess_new_air(df)
    concatenated_df = pd.concat([old_df, new_df], axis=0, ignore_index=True)
    return concatenated_df

In [26]:
test_df = find_mel_1hr_data('data_sets/air_data_realtime.csv')
updated_df = concate_new_with_old(test_df, df_mel2)
updated_df.to_csv('data_sets/air_static.csv') 

FileNotFoundError: [Errno 2] No such file or directory: 'air_data_realtime.csv'